In [ ]:
try:
    passes += 1
    !git pull
except:
    passes = 1
    !git clone https://github.com/arminwitte/FoolsUNet.git
    %cd FoolsUNet

In [ ]:
import tensorflow as tf

print(tf.__version__)

### Data Preparation

In [1]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # strategy = tf.distribute.get_strategy()
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

NameError: name 'pr' is not defined

## Get pre-trained encoder and attach classification head

In [ ]:
from foolsunet.model import foolsunet

In [ ]:
with strategy.scope():
    base_model = tf.keras.models.load_model("foolsunet_imagenet_resized_0-5_epoch.keras")
    #base_model = foolsunet()
    base_model.summary()

### Define encoder

In [ ]:
# Use the activations of these layers
layer_names = [
    #'block_1_downsample',   # 128x128
    #'block_2_downsample',   # 64x64
    'block_3_invres_downsample',   # 32x32
    'block_4_invres_downsample',  # 16x16 
    'block_5_invres_downsample',      # 8x8
    'block_6_invres_downsample', # 4x4
]
base_model_layers = [base_model.get_layer(name) for name in layer_names]

# # Create the feature extraction model
# down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

# down_stack.trainable = False

### Define classifier

In [ ]:
def assemble_model():
    inputs = tf.keras.layers.Input(shape=[64, 64, 3])
    x = inputs
#     x = base_model.input
#     x = down_stack(inputs)
    x = tf.keras.layers.Conv2D(128,1,activation='relu')(x)
    for layer in base_model_layers:
#         layer.trainable = False
        x = layer(x)
    x = tf.keras.layers.Conv2D(1024, 1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Activation("relu6")(x)
    x = tf.keras.layers.AveragePooling2D(4)(x)
    x = tf.keras.layers.Conv2D(num_classes, 1)(x)
    x = tf.keras.layers.Flatten(name="outputs")(x)

    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
num_classes = 1000
# classification_head = tf.keras.Sequential([
#     tf.keras.layers.Input([8,8,128]),
#     tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(num_classes, name="outputs"),])
# # classification_head.build()

In [ ]:
# model = tf.keras.Sequential([base_model.get_layer(name) for name in layer_names] + [layers.Flatten(),
#   layers.Dense(128, activation='relu'),
#   layers.Dense(num_classes, name="outputs"),
#                                                                                    ])
# model.build()

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model("foolsunet_imagenet_resized_0-5_epoch.keras")
    # model = assemble_model()
    opt = tf.keras.optimizers.Adam(learning_rate=0.005)
    model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
    model.summary()


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

### Get Dataset

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
dataset, info = tfds.load("imagenet_resized/64x64:0.*.*", with_info=True, data_dir="/kaggle/tmp/")

In [ ]:
dataset

In [ ]:
info

In [ ]:
from tensorflow.keras import layers
class Augment(layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = layers.RandomFlip(mode="horizontal", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        return inputs, labels

In [ ]:
def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

In [ ]:
def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (64, 64))
    input_label = datapoint["label"]

    input_image = normalize(input_image)

    return input_image, input_label

In [ ]:
TRAIN_LENGTH = info.splits["train"].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train_images = dataset["train"].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset["validation"].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_batches = (
    train_images#.cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    #.map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_batches = test_images.batch(BATCH_SIZE)

### Train Model

In [ ]:
EPOCHS = 5
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits["validation"].num_examples // BATCH_SIZE // VAL_SUBSPLITS

model_history = model.fit(
    train_batches,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    validation_data=test_batches,
    # callbacks=[DisplayCallback()]
)

### Save Model 

In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save("foolsunet_imagenet_resized.keras")

In [ ]:

model.save_weights("foolsunet_imagenet_resized.weights.h5")

In [ ]:
new_model = tf.keras.models.load_model("foolsunet_imagenet_resized.keras")

# Show the model architecture
new_model.summary()